# Simple plotting notebook to test overlay

Be careful, before running this notebook, the SLVNA Python server has to be stopped.
It starts every minute unless you edit root's crontab (via SSH) with `crontab -e -u root`
and comment the entry with start_vna_server.sh. To stop the Python server manually,
run via SSH `nc localhost 2024`, then type ! followed by ctrl + d and finally ctrl + c.
Also make sure the Red Pitaya has a clock signal (either external or internal), otherwise
the DMA request will hang.

In [1]:
import numpy as np
from pynq import MMIO, Overlay, allocate
import matplotlib.pyplot as plt
import time

## Helper functions

In [2]:
RAW_TO_VOLTS = 2**-25   # Fixed point to floating point

def uint64_to_signed_int(unsigned):
    """Converts 64-bit unsigned integer to signed integer. By Bit Twiddling Hacks; see
    https://stackoverflow.com/questions/1375897/how-to-get-the-signed-integer-value-of-a-long-in-python.
    """
    unsigned &= (1<<64) - 1  # Keep only the lowest 64 bits.
    return (unsigned ^ (1<<63)) - (1<<63)  # Swap and shift down.

def buffer_to_volts(buffer):
    """Divides integer I and Q values by sample count. Buffer is an array containing a multiple of three
    elements: I value, Q value, count. The I and Q values are divided by count
    and multiplied by a conversion factor to get the unit of volts.
    """
    volts = [
        (
            uint64_to_signed_int(int(buffer[i])          # to Python integer(first entry: unsigned 32-bit integer
            + (int(buffer[i + 1]) << 32))                #     adding second unsigned integer shifted left 32 bits)
            / buffer[i + 2]                              # dividing by third entry (count)
            * RAW_TO_VOLTS                               # scaling to units of volts
        )
        for i in range(0, 12, 3)                         # i = 0, 3, 6, 9
    ]
    return volts

def volts_to_phasors(volts):
    """Interpret the 4 voltage values as phasors
    """
    dut = volts[0] + 1j*volts[1]
    ref = volts[2] + 1j*volts[3]
    rel = dut/ref
    return dut, ref, rel

In [3]:
# Set configuration bits using MMIO

ADC_FREQ = 125_000_000
cyc = lambda x: round(x*ADC_FREQ)
RESET_BIT = 0b1

# Trigger configuration, eg trigger1_conf = TRIG_POS+TRIG_SWEEP
TRIG_POS = 0b0000
TRIG_NEG = 0b0001
TRIG_SWEEP = 0b0010
TRIG_POINTS = 0b0100

mmio_dead_time = MMIO(0x42000000)       # Dead time in ADC samples
mmio_point_time = MMIO(0x42000008)      # Total point time in ADC samples (dead time + accumulation time)
mmio_trigger_conf = MMIO(0x41200000)    # Trigger config
mmio_general_conf = MMIO(0x41200008)    # General config

def set_config(dead_time = 300E-6, point_time = 1E-3, trigger_length = 10E-6, trigger1_conf = 0, trigger2_conf = 0):
#     assert trigger_length <= dead_time <= point_time, "Trigger length should be less than settling time, less than total point time!"
    
    mmio_dead_time.write(0, cyc(dead_time))              # Only one value in register, overwrite completely
    mmio_point_time.write(0, cyc(point_time))            # Idem
    mmio_trigger_conf.write(0, cyc(trigger_length) + (trigger1_conf << 24) + (trigger2_conf << 28))

def read_status():
    return mmio_general_conf.read(0)

def start_acq(dma_recv):
    curr = mmio_general_conf.read(0)
    mmio_general_conf.write(0, curr | RESET_BIT)
    
    # dma request of 16 words to get rid of misformed packet
    buffer = allocate(shape=(16,), dtype=np.uint32)
    dma_recv.transfer(buffer)
    dma_recv.wait()
    del buffer

def stop_acq():
    curr = mmio_general_conf.read(0)
    mmio_general_conf.write(0, curr & ~RESET_BIT)

## Using the VNA

In [ ]:
# Program the overlay onto the PL, configure it and get a handle for the DMA

ol = Overlay("/home/xilinx/bit/slvna_v1_0.bit")

dma = ol.dma
dma_recv = dma.recvchannel

In [ ]:
# Allocate input buffer and show it's empty

data_size = 12
buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(data_size):
    print(f'0x{format(buffer[i], "02x")}, ', end='')

In [ ]:
# Transfer data and show it (can repeat indefinitely)

set_config(dead_time=250E-6, point_time=1E-3, trigger_length=10E-6, trigger1_conf=TRIG_POS+TRIG_SWEEP, trigger2_conf=TRIG_POS+TRIG_POINTS)

# Acquire data
start_acq(dma_recv)
dma_recv.transfer(buffer)
dma_recv.wait()
stop_acq()

# Convert it to human units
volts = buffer_to_volts(buffer)
dut, ref, rel = volts_to_phasors(volts)

# Print results
for i in range(data_size):
    print(f'0x{format(buffer[i], "02x")}, ', end='')
print()
print(volts)
print(f'DUT: {np.abs(dut):.3f}V, {np.angle(dut):.3f}r, ' +
      f'REF: {np.abs(ref):.3f}V, {np.angle(ref):.3f}r, ' +
      f'REL: {np.abs(rel):.3f}, {np.angle(rel):.3f}r')

In [ ]:
POINTS = 4301

values = np.ndarray((POINTS, data_size), dtype=np.uint32)
set_config(dead_time=900E-6, point_time=1E-3, trigger_length=10E-6, trigger1_conf=TRIG_POS+TRIG_POINTS+TRIG_SWEEP, trigger2_conf=TRIG_POS+TRIG_POINTS)

cnt = 0
start_acq(dma_recv)
start = time.perf_counter()
for i in range(POINTS):
    dma_recv.transfer(buffer)
    dma_recv.wait()
    values[i] = buffer
end = time.perf_counter()
stop_acq()
print(f"Took {end-start:.4f}s")

# Calculate after the loop to allow faster IFBWs (above 2k instead of barely 1k)
rel_values = []
for buf in values:
    volts = buffer_to_volts(buf)
    dut, ref, rel = volts_to_phasors(volts)
    rel_values.append(rel)

plt.plot(np.abs(rel_values))
# plt.ylim(0, 1)
plt.show()
plt.plot(np.angle(rel_values))
# plt.ylim(-np.pi, np.pi)
plt.show()

In [25]:
stop_acq()